# Demo - Deployment Test

The following notebook allows us to manually test a model deployment endpoint with a test image and visulaise the result

## Set Model Endpoint 
Choose which endpoint that will be tested

### Define Endpoint
Set the AML endpoint to test

In [1]:
# endpoint url is found in the AML studio. You must append the probaility argument in the form url/score?prob=0.5
endpoint = ''

enable_auth = True
auth = ''

# Test image path
test_img = f'./test_image.jpg'

## Load Image

In [ ]:
img = open(test_img, 'rb').read()

## Score Image

In [ ]:
# Call the endpoint
import requests
import json

headers = { 'Content-Type':'application/json' }

if enable_auth:
    headers['Authorization']=f'Bearer {auth}'

resp = requests.post(endpoint, img, headers=headers)
print(resp)
results = resp.text
print(results)

## Present Results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
import numpy as np

In [ ]:
img_np=mpimg.imread(test_img)
img = Image.fromarray(img_np.astype('uint8'),'RGB')
x, y = img.size
IMAGE_SIZE = (x/50,y/50)
plt.figure(figsize=IMAGE_SIZE)

### Plot Result

In [ ]:
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(img_np)

# draw box and label for each detection 
detections = json.loads(results)
for detect in detections:
    label = detect['label']
    box = detect['bounding_box']
    ymin, xmin, ymax, xmax = box[0], box[1], box[2], box[3]
    topleft_x, topleft_y = x * xmin, y * ymin
    width, height = x * (xmax - xmin), y * (ymax - ymin)
    print('{}: {}, {}, {}, {}'.format(detect['label'], topleft_x, topleft_y, width, height))

    color = 'white'
    rect = patches.Rectangle((topleft_x, topleft_y), width, height, 
                             linewidth=2, edgecolor=color,facecolor='none')

    ax.add_patch(rect)
    plt.text(topleft_x, topleft_y, label, color=color)

plt.show()

### Plot Mask Result

For mask_rcnn based models, the model generates for each input image multiple masks of fixed sizes. One needs to resize the masks to its original shape, put them into the right positions and generate contour polygons from them.

In [ ]:
import cv2
from skimage.transform import resize
import PIL.ImageDraw as ImageDraw

def unmold_mask(mask, bbox, image_shape):
    """
    Converts a mask generated by the neural network to a format similar to its original shape
    
    mask: [height, width] of type float. A small, typically 28x28 mask.
    bbox: [y1, x1, y2, x2]. The box to fit the mask in.
    Returns a binary mask with the same size as the original image.
    """
    threshold = 0.5
    y1, x1, y2, x2 = bbox
    mask = resize(mask, (y2 - y1, x2 - x1))
    mask = np.where(mask >= threshold, 1, 0).astype(np.bool)

    # Put the mask in the right location.
    full_mask = np.zeros(image_shape[:2], dtype=np.bool)
    full_mask[y1:y2, x1:x2] = mask
    return full_mask


def visualize_polygon(img_path, polygons, color=(255,0,0,128)):
    img = Image.open(img_path)
    img2 = img.copy()
    draw = ImageDraw.Draw(img2)
    for polygon in polygons:
        draw.polygon(tuple(polygon), fill=color)
    img3 = Image.blend(img, img2, 0.5)
    img3.show()

In [ ]:
# draw polygons for each detection
detections = json.loads(results)
polygons = []
for detect in detections:
    label = detect['label']
    box = detect['bounding_box']
    ymin, xmin, ymax, xmax = box[0], box[1], box[2], box[3]
    topleft_x, topleft_y, width, height = x * xmin, y * ymin, x * (xmax - xmin), y * (ymax - ymin)  # De-normalizing

    # extract and resize the predicted mask
    mask = detect['mask']
    mask_np = np.array(mask)
    mask = unmold_mask(mask=mask_np, bbox=[int(topleft_y), int(topleft_x), int(topleft_y)+int(height), int(topleft_x)+int(width)], image_shape=(y,x))
    mask_img = Image.fromarray((mask * 255).astype(np.uint8))
    mask_img.show()
    
    # extract segmentation polygons based on mask contour
    contours, _ = cv2.findContours((mask * 255).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if contour.size > 6:  # Need at least 6 (3 points) to build a valid polygon
            polygon = contour.flatten().tolist()  # Extract polygon list
            polygons.append(polygon)

visualize_polygon(img_path=test_img, polygons=polygons, color=(0, 255, 0, 255))